In [12]:
## TESTING Deep learning

In [1]:
import pandas as pd
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.layers import concatenate
# Load the dataset
final_dataset_balanced = pd.read_parquet('../Data/Data/FinalDataSet/Date_final_dataset_balanced_float32.parquet')


In [2]:
wf_df_train=final_dataset_balanced[final_dataset_balanced.acq_date<'2022-01-01']
wf_df_valid=final_dataset_balanced[final_dataset_balanced.acq_date>='2022-01-01']

In [3]:
for c in wf_df_train.columns:
    print(c)

acq_date
latitude
longitude
is_holiday
day_of_week
day_of_year
is_weekend
NDVI
SoilMoisture
sea_distance
station_lat
station_lon
average_temperature_lag_1
average_temperature_lag_2
average_temperature_lag_3
average_temperature_lag_4
average_temperature_lag_5
average_temperature_lag_6
average_temperature_lag_7
average_temperature_lag_8
average_temperature_lag_9
average_temperature_lag_10
average_temperature_lag_11
average_temperature_lag_12
average_temperature_lag_13
average_temperature_lag_14
average_temperature_lag_15
maximum_temperature_lag_1
maximum_temperature_lag_2
maximum_temperature_lag_3
maximum_temperature_lag_4
maximum_temperature_lag_5
maximum_temperature_lag_6
maximum_temperature_lag_7
maximum_temperature_lag_8
maximum_temperature_lag_9
maximum_temperature_lag_10
maximum_temperature_lag_11
maximum_temperature_lag_12
maximum_temperature_lag_13
maximum_temperature_lag_14
maximum_temperature_lag_15
minimum_temperature_lag_1
minimum_temperature_lag_2
minimum_temperature_lag_3
m

In [4]:
# Balance the dataset by taking the same number of positive and negative samples
# wf_df_train = wf_df_train.sample(frac=1)

# Get the minimum number of samples in each class
min_samples = min(wf_df_train['is_fire'].value_counts())

# Balance the training dataset
wf_df_train_balanced = wf_df_train.groupby('is_fire').apply(lambda x: x.sample(min_samples)).reset_index(drop=True)

# Balance the validation dataset
min_samples_valid = min(wf_df_valid['is_fire'].value_counts())
wf_df_valid_balanced = wf_df_valid.groupby('is_fire').apply(lambda x: x.sample(min_samples_valid)).reset_index(drop=True)

wf_df_train_balanced = wf_df_train_balanced.sample(frac=1)
wf_df_valid_balanced = wf_df_valid_balanced.sample(frac=1)

# removing dates
acq_date_train=wf_df_train_balanced.pop('acq_date')
acq_date_valid=wf_df_valid_balanced.pop('acq_date')


In [7]:
wf_df_valid_balanced

,latitude,longitude,is_holiday,day_of_week,day_of_year,is_weekend,NDVI,SoilMoisture,sea_distance,station_lat,...,wind_gust_quarterly_mean,dew_point_quarterly_mean,average_temperature_yearly_mean,maximum_temperature_yearly_mean,minimum_temperature_yearly_mean,precipitation_yearly_mean,snow_depth_yearly_mean,wind_gust_yearly_mean,dew_point_yearly_mean,is_fire
7834,28.439558,-10.029773,1.0,6.0,310.0,1.0,707.0,12.0,92553.781250,29.017000,...,787.164124,59.295650,69.822739,79.701370,84.747673,2.744904,999.900024,183.254242,49.621368,0.0
33887,35.050900,-5.728900,0.0,3.0,195.0,0.0,4378.0,15.5,57893.476562,35.150002,...,837.813171,56.591209,66.166298,71.470276,57.671505,0.348274,999.900024,114.576988,56.370411,1.0
38003,35.269577,-5.109628,0.0,4.0,196.0,0.0,5743.0,49.0,43970.628906,35.167000,...,999.900024,52.151649,65.949043,76.781372,51.520000,0.066507,999.900024,999.900024,242.498215,1.0
15305,31.672754,-7.460205,0.0,1.0,263.0,0.0,1882.0,17.0,224915.640625,31.607000,...,613.298889,51.762638,69.567673,84.606300,56.657536,0.012178,999.900024,864.437256,49.907806,0.0
18590,32.170471,-8.861309,1.0,1.0,123.0,0.0,2726.0,17.0,58737.328125,32.283001,...,988.934448,48.271667,68.063423,75.906578,59.228493,0.029315,999.900024,989.070435,56.575615,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23178,35.261955,-5.505293,0.0,6.0,212.0,1.0,5572.0,20.0,67425.273438,35.167000,...,999.900024,52.151649,65.949043,76.781372,51.520000,0.066507,999.900024,999.900024,242.498215,1.0
37663,35.280132,-6.061341,0.0,3.0,195.0,0.0,3283.0,29.5,11754.611328,35.150002,...,837.813171,56.591209,66.166298,71.470276,57.671505,0.348274,999.900024,114.576988,56.370411,1.0
6284,31.225174,-6.009823,0.0,1.0,46.0,0.0,1123.0,21.0,372666.875000,30.938999,...,946.883667,27.145653,69.711647,82.831917,55.054520,0.286767,999.900024,871.874268,26.710958,0.0
2183,32.077972,-6.991622,0.0,0.0,353.0,0.0,2495.0,31.0,222927.562500,32.367001,...,905.444580,50.809784,70.802742,82.505203,108.412605,4.931863,999.900024,976.021912,46.451508,0.0


In [8]:

# # Split the dataset into features and target
# X = final_dataset_balanced.drop(columns=['is_fire'])
# y = final_dataset_balanced['is_fire']

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize a StandardScaler
# scaler = StandardScaler()

# # Fit the scaler on the training features and transform both the training and testing features
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Split the training dataset into features and target
y_train = wf_df_train_balanced['is_fire']
X_train = wf_df_train_balanced.drop(columns=['is_fire'])

# Split the validation dataset into features and target
y_valid = wf_df_valid_balanced['is_fire']
X_valid = wf_df_valid_balanced.drop(columns=['is_fire'])

# Initialize a StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training features and transform both the training and validation features
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

## FFN POS

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Layer, LayerNormalization, Embedding, Flatten, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Assuming final_dataset_balanced and wf_df_train_balanced, wf_df_valid_balanced are predefined

# Split the dataset into features and target
X = final_dataset_balanced.drop(columns=['is_fire'])
y = final_dataset_balanced['is_fire']

# Remove DateTime columns (if any)
datetime_columns = X.select_dtypes(include=['datetime64']).columns
X = X.drop(columns=datetime_columns)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training features and transform both the training and testing features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Split the training dataset into features and target
X_train = wf_df_train_balanced.drop(columns=['is_fire'])
y_train = wf_df_train_balanced['is_fire']

# Split the validation dataset into features and target
X_valid = wf_df_valid_balanced.drop(columns=['is_fire'])
y_valid = wf_df_valid_balanced['is_fire']

# Remove DateTime columns (if any)
datetime_columns_train = X_train.select_dtypes(include=['datetime64']).columns
X_train = X_train.drop(columns=datetime_columns_train)
X_valid = X_valid.drop(columns=datetime_columns_train)

# Initialize a StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training features and transform both the training and validation features
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# FFN with Positional Encoding
class FFNWithPosEncoding(Layer):
    def __init__(self, num_columns, embed_dim, ff_dim, rate=0.1):
        super(FFNWithPosEncoding, self).__init__()
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)  # Output matches embed_dim
        ])
        self.layernorm = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(rate)
        self.pos_emb = Embedding(input_dim=num_columns, output_dim=embed_dim)
        self.positions = tf.range(start=0, limit=num_columns, delta=1)

    def call(self, inputs, training=False):
        # Expand dimensions of inputs to match embedding output
        x = tf.expand_dims(inputs, -1)

        # Pass inputs through the feed forward network
        x = self.ffn(x)

        # Get positional embeddings
        pos_encoding = self.pos_emb(self.positions)

        # Expand pos_encoding to match the batch size of inputs
        pos_encoding = tf.expand_dims(pos_encoding, 0)
        pos_encoding = tf.tile(pos_encoding, [tf.shape(inputs)[0], 1, 1])

        # Add dropout
        x = self.dropout(x, training=training)

        # Add positional encodings
        x += pos_encoding

        # Apply layer normalization
        return self.layernorm(x)

# Model configuration
num_columns = X_train_scaled.shape[1]
embed_dim = 32  # Embedding size for each token
ff_dim = 32  # Hidden layer size in feed forward network

inputs = Input(shape=(num_columns,))
x = FFNWithPosEncoding(num_columns, embed_dim, ff_dim)(inputs)
x = Flatten()(x)  # Flatten the output to align with the output layer
outputs = Dense(1, activation="sigmoid")(x)  # Single output neuron for binary classification
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=128, validation_data=(X_valid_scaled, y_valid), verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Accuracy: {accuracy}")

# Predict using the trained model
predicted_prob = model.predict(X_test_scaled)
predicted_class = (predicted_prob > 0.5).astype(int).reshape(-1)

# Compare hardcoded labels and predicted labels
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predicted_class
})

# Calculate the mean difference (not very meaningful for binary classification, but for demonstration)
mean_difference = np.mean(np.abs(y_test - predicted_class))
print(f"Mean Difference: {mean_difference}")

# Display the comparison
print(comparison_df.head())


# embeding layer for continoues values

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Layer, LayerNormalization, Embedding, Flatten, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer

# Assuming final_dataset_balanced and wf_df_train_balanced, wf_df_valid_balanced are predefined

# Split the dataset into features and target
X = final_dataset_balanced.drop(columns=['is_fire'])
y = final_dataset_balanced['is_fire']

# Remove DateTime columns (if any)
datetime_columns = X.select_dtypes(include=['datetime64']).columns
X = X.drop(columns=datetime_columns)

# Discretize continuous features
discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
X_discretized = discretizer.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_discretized, y, test_size=0.2, random_state=42)

# Split the training dataset into features and target
X_train = wf_df_train_balanced.drop(columns=['is_fire'])
y_train = wf_df_train_balanced['is_fire']

# Split the validation dataset into features and target
X_valid = wf_df_valid_balanced.drop(columns=['is_fire'])
y_valid = wf_df_valid_balanced['is_fire']

# Remove DateTime columns (if any)
datetime_columns_train = X_train.select_dtypes(include=['datetime64']).columns
X_train = X_train.drop(columns=datetime_columns_train)
X_valid = X_valid.drop(columns=datetime_columns_train)

# Discretize continuous features
X_train_discretized = discretizer.fit_transform(X_train)
X_valid_discretized = discretizer.transform(X_valid)

# FFN with Positional Encoding
class FFNWithPosEncoding(Layer):
    def __init__(self, num_columns, embed_dim, ff_dim, rate=0.1):
        super(FFNWithPosEncoding, self).__init__()
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)  # Output matches embed_dim
        ])
        self.layernorm = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(rate)
        self.pos_emb = Embedding(input_dim=num_columns, output_dim=embed_dim)
        self.positions = tf.range(start=0, limit=num_columns, delta=1)

    def call(self, inputs, training=False):
        # Expand dimensions of inputs to match embedding output
        x = tf.expand_dims(inputs, -1)

        # Pass inputs through the feed forward network
        x = self.ffn(x)

        # Get positional embeddings
        pos_encoding = self.pos_emb(self.positions)

        # Expand pos_encoding to match the batch size of inputs
        pos_encoding = tf.expand_dims(pos_encoding, 0)
        pos_encoding = tf.tile(pos_encoding, [tf.shape(inputs)[0], 1, 1])

        # Add dropout
        x = self.dropout(x, training=training)

        # Add positional encodings
        x += pos_encoding

        # Apply layer normalization
        return self.layernorm(x)

# Model configuration
num_columns = X_train_discretized.shape[1]
embed_dim = 32  # Embedding size for each token
ff_dim = 32  # Hidden layer size in feed forward network

inputs = Input(shape=(num_columns,))
x = FFNWithPosEncoding(num_columns, embed_dim, ff_dim)(inputs)
x = Flatten()(x)  # Flatten the output to align with the output layer
outputs = Dense(1, activation="sigmoid")(x)  # Single output neuron for binary classification
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_discretized, y_train, epochs=10, batch_size=128, validation_data=(X_valid_discretized, y_valid), verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy}")

# Predict using the trained model
predicted_prob = model.predict(X_test)
predicted_class = (predicted_prob > 0.5).astype(int).reshape(-1)

# Compare hardcoded labels and predicted labels
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predicted_class
})

# Calculate the mean difference (not very meaningful for binary classification, but for demonstration)
mean_difference = np.mean(np.abs(y_test - predicted_class))
print(f"Mean Difference: {mean_difference}")

# Display the comparison
print(comparison_df.head())


c:\DevLib\Conda\envs\TF\lib\site-packages\sklearn\preprocessing\_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
c:\DevLib\Conda\envs\TF\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 227 is constant and will be replaced with 0.
  warnings.warn(
c:\DevLib\Conda\envs\TF\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 255 is constant and will be replaced with 0.
  warnings.warn(
c:\DevLib\Conda\envs\TF\lib\site-packages\sklearn\preprocessing\_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
c:\DevLib\Conda\envs\TF\lib\site-packages\

Epoch 1/10
6471/6471 [==============================] - 36s 6ms/step - loss: 0.4199 - accuracy: 0.8156 - val_loss: 0.3223 - val_accuracy: 0.8571
Epoch 2/10
6471/6471 [==============================] - 32s 5ms/step - loss: 0.3934 - accuracy: 0.8307 - val_loss: 0.3219 - val_accuracy: 0.8595
Epoch 3/10
6471/6471 [==============================] - 36s 5ms/step - loss: 0.3887 - accuracy: 0.8327 - val_loss: 0.3273 - val_accuracy: 0.8528
Epoch 4/10
6471/6471 [==============================] - 34s 5ms/step - loss: 0.3834 - accuracy: 0.8355 - val_loss: 0.3190 - val_accuracy: 0.8642
Epoch 5/10
6471/6471 [==============================] - 35s 5ms/step - loss: 0.3792 - accuracy: 0.8370 - val_loss: 0.3484 - val_accuracy: 0.8507
Epoch 6/10
6471/6471 [==============================] - 34s 5ms/step - loss: 0.3766 - accuracy: 0.8383 - val_loss: 0.2946 - val_accuracy: 0.8632
Epoch 7/10
6471/6471 [==============================] - 33s 5ms/step - loss: 0.3746 - accuracy: 0.8394 - val_loss: 0.3153 - val_ac

# FFN _ POS NO EMBEDING

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Layer, LayerNormalization, Flatten, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming final_dataset_balanced and wf_df_train_balanced, wf_df_valid_balanced are predefined

# Split the dataset into features and target
X = final_dataset_balanced.drop(columns=['is_fire'])
y = final_dataset_balanced['is_fire']

# Remove DateTime columns (if any)
datetime_columns = X.select_dtypes(include=['datetime64']).columns
X = X.drop(columns=datetime_columns)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training features and transform both the training and testing features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# FFN with Positional Encoding
class FFNWithPosEncoding(Layer):
    def __init__(self, num_features, ff_dim, rate=0.1):
        super(FFNWithPosEncoding, self).__init__()
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(num_features)  # Output matches the number of features
        ])
        self.layernorm = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(rate)

    def call(self, inputs, training=False):
        # Pass inputs through the feed forward network
        x = self.ffn(inputs)

        # Calculate positional encoding
        position_encoding = self.get_positional_encoding(tf.shape(inputs)[0], tf.shape(inputs)[1])

        # Add positional encodings
        x += position_encoding

        # Apply dropout
        x = self.dropout(x, training=training)

        # Apply layer normalization
        return self.layernorm(x)

    def get_positional_encoding(self, batch_size, num_features):
        position_indices = tf.range(num_features, dtype=tf.float32)
        position_encoding = position_indices[tf.newaxis, :]  # Shape: (1, num_features)
        position_encoding = tf.tile(position_encoding, [batch_size, 1])  # Match batch size
        return position_encoding

# Model configuration
num_features = X_train_scaled.shape[1]
ff_dim = 32  # Hidden layer size in feed forward network

inputs = Input(shape=(num_features,))
x = FFNWithPosEncoding(num_features, ff_dim)(inputs)
x = Flatten()(x)  # Flatten the output to align with the output layer
outputs = Dense(1, activation="sigmoid")(x)  # Single output neuron for binary classification
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=128, validation_data=(X_test_scaled, y_test), verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Accuracy: {accuracy}")

# Predict using the trained model
predicted_prob = model.predict(X_test_scaled)
predicted_class = (predicted_prob > 0.5).astype(int).reshape(-1)

# Compare hardcoded labels and predicted labels
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predicted_class
})

# Display the comparison
print(comparison_df.head())


Epoch 1/100
5842/5842 [==============================] - 25s 4ms/step - loss: 0.3922 - accuracy: 0.8235 - val_loss: 0.3426 - val_accuracy: 0.8562
Epoch 2/100
5842/5842 [==============================] - 25s 4ms/step - loss: 0.3280 - accuracy: 0.8618 - val_loss: 0.3079 - val_accuracy: 0.8716
Epoch 3/100
5842/5842 [==============================] - 26s 5ms/step - loss: 0.3020 - accuracy: 0.8747 - val_loss: 0.2912 - val_accuracy: 0.8800
Epoch 4/100
5842/5842 [==============================] - 26s 4ms/step - loss: 0.2860 - accuracy: 0.8824 - val_loss: 0.2798 - val_accuracy: 0.8856
Epoch 5/100
5842/5842 [==============================] - 26s 4ms/step - loss: 0.2751 - accuracy: 0.8878 - val_loss: 0.2668 - val_accuracy: 0.8920
Epoch 6/100
5842/5842 [==============================] - 26s 4ms/step - loss: 0.2666 - accuracy: 0.8915 - val_loss: 0.2618 - val_accuracy: 0.8940
Epoch 7/100
5842/5842 [==============================] - 26s 4ms/step - loss: 0.2603 - accuracy: 0.8947 - val_loss: 0.2555 -

## Equivalat simple straitforword FFN

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Flatten, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming final_dataset_balanced and wf_df_train_balanced, wf_df_valid_balanced are predefined

# Split the dataset into features and target
X = final_dataset_balanced.drop(columns=['is_fire'])
y = final_dataset_balanced['is_fire']

# Remove DateTime columns (if any)
datetime_columns = X.select_dtypes(include=['datetime64']).columns
X = X.drop(columns=datetime_columns)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training features and transform both the training and testing features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the simple FFN model
num_features = X_train_scaled.shape[1]
ff_dim = 32  # Hidden layer size in feed forward network

inputs = Input(shape=(num_features,))
x = Dense(ff_dim, activation="relu")(inputs)
x = Dense(num_features, activation="relu")(x)  # Match the number of output dimensions as the original FFN
x = LayerNormalization(epsilon=1e-6)(x)
x = Dropout(0.1)(x)
x = Flatten()(x)
outputs = Dense(1, activation="sigmoid")(x)  # Single output neuron for binary classification
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=128, validation_data=(X_test_scaled, y_test), verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Accuracy: {accuracy}")

# Predict using the trained model
predicted_prob = model.predict(X_test_scaled)
predicted_class = (predicted_prob > 0.5).astype(int).reshape(-1)

# Compare hardcoded labels and predicted labels
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predicted_class
})

# Display the comparison
print(comparison_df.head())


Epoch 1/100
5842/5842 [==============================] - 26s 4ms/step - loss: 0.3231 - accuracy: 0.8612 - val_loss: 0.2579 - val_accuracy: 0.8931
Epoch 2/100
5842/5842 [==============================] - 24s 4ms/step - loss: 0.2437 - accuracy: 0.9001 - val_loss: 0.2159 - val_accuracy: 0.9130
Epoch 3/100
5842/5842 [==============================] - 24s 4ms/step - loss: 0.2114 - accuracy: 0.9157 - val_loss: 0.1959 - val_accuracy: 0.9223
Epoch 4/100
5842/5842 [==============================] - 24s 4ms/step - loss: 0.1913 - accuracy: 0.9258 - val_loss: 0.1777 - val_accuracy: 0.9333
Epoch 5/100
5842/5842 [==============================] - 25s 4ms/step - loss: 0.1766 - accuracy: 0.9330 - val_loss: 0.1643 - val_accuracy: 0.9396
Epoch 6/100
5842/5842 [==============================] - 24s 4ms/step - loss: 0.1666 - accuracy: 0.9380 - val_loss: 0.1558 - val_accuracy: 0.9451
Epoch 7/100
5842/5842 [==============================] - 24s 4ms/step - loss: 0.1587 - accuracy: 0.9419 - val_loss: 0.1487 -

## Trasformer

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Flatten, Input, MultiHeadAttention, Add
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer

# Assuming final_dataset_balanced and wf_df_train_balanced, wf_df_valid_balanced are predefined

# Split the dataset into features and target
X = final_dataset_balanced.drop(columns=['is_fire'])
y = final_dataset_balanced['is_fire']

# Remove DateTime columns (if any)
datetime_columns = X.select_dtypes(include=['datetime64']).columns
X = X.drop(columns=datetime_columns)

# Discretize continuous features
discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
X_discretized = discretizer.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_discretized, y, test_size=0.2, random_state=42)

# Split the training dataset into features and target
X_train = wf_df_train_balanced.drop(columns=['is_fire'])
y_train = wf_df_train_balanced['is_fire']

# Split the validation dataset into features and target
X_valid = wf_df_valid_balanced.drop(columns=['is_fire'])
y_valid = wf_df_valid_balanced['is_fire']

# Remove DateTime columns (if any)
datetime_columns_train = X_train.select_dtypes(include=['datetime64']).columns
X_train = X_train.drop(columns=datetime_columns_train)
X_valid = X_valid.drop(columns=datetime_columns_train)

# Discretize continuous features
X_train_discretized = discretizer.fit_transform(X_train)
X_valid_discretized = discretizer.transform(X_valid)

# Positional Encoding
class PositionalEncoding(Layer):
    def __init__(self, num_columns, embed_dim):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(num_columns, embed_dim)
    
    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates
    
    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(np.arange(position)[:, np.newaxis],
                                     np.arange(d_model)[np.newaxis, :],
                                     d_model)
        
        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        
        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        
        pos_encoding = angle_rads[np.newaxis, ...]
        
        return tf.cast(pos_encoding, dtype=tf.float32)
    
    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

# Transformer Block
def transformer_block(inputs, num_heads, ff_dim, dropout_rate):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attn_output]))
    
    ffn_output = Dense(ff_dim, activation='relu')(out1)
    ffn_output = Dense(inputs.shape[-1])(ffn_output)
    ffn_output = Dropout(dropout_rate)(ffn_output)
    return LayerNormalization(epsilon=1e-6)(Add()([out1, ffn_output]))

# Model configuration
num_columns = X_train_discretized.shape[1]
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network
dropout_rate = 0.1

inputs = Input(shape=(num_columns,))
x = Dense(embed_dim)(inputs)
x = PositionalEncoding(num_columns, embed_dim)(x)

# Add multiple transformer blocks
num_transformer_blocks = 2
for _ in range(num_transformer_blocks):
    x = transformer_block(x, num_heads, ff_dim, dropout_rate)

x = Flatten()(x)  # Flatten the output to align with the output layer
outputs = Dense(1, activation="sigmoid")(x)  # Single output neuron for binary classification
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_discretized, y_train, epochs=10, batch_size=128, validation_data=(X_valid_discretized, y_valid), verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy}")

# Predict using the trained model
predicted_prob = model.predict(X_test)
predicted_class = (predicted_prob > 0.5).astype(int).reshape(-1)

# Compare hardcoded labels and predicted labels
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predicted_class
})

# Calculate the mean difference (not very meaningful for binary classification, but for demonstration)
mean_difference = np.mean(np.abs(y_test - predicted_class))
print(f"Mean Difference: {mean_difference}")

# Display the comparison
print(comparison_df.head())


c:\DevLib\Conda\envs\TF\lib\site-packages\sklearn\preprocessing\_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
c:\DevLib\Conda\envs\TF\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 227 is constant and will be replaced with 0.
  warnings.warn(
c:\DevLib\Conda\envs\TF\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 255 is constant and will be replaced with 0.
  warnings.warn(
c:\DevLib\Conda\envs\TF\lib\site-packages\sklearn\preprocessing\_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
c:\DevLib\Conda\envs\TF\lib\site-packages\

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'model_2/positional_encoding/add' defined at (most recent call last):
    File "c:\DevLib\Conda\envs\TF\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\DevLib\Conda\envs\TF\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\DevLib\Conda\envs\TF\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\DevLib\Conda\envs\TF\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\DevLib\Conda\envs\TF\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "c:\DevLib\Conda\envs\TF\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\DevLib\Conda\envs\TF\lib\asyncio\windows_events.py", line 321, in run_forever
      super().run_f

# Models Definitions

In [7]:
def create_minimum_ffn(input_dim, output_dim,normalizer,scale=1,midlel_function='sigmoid' ):
    # Clear the session
    K.clear_session()
    layer1_size=16
    # Create the model
    # Input layer
    input_layer = Input(shape=(input_dim,))

    # Batch normalization for input
    normalized_input = input_layer

    x = Dense(layer1_size*scale)(normalized_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.7)(x)

    x = Dense(8*scale)(normalized_input)
    x = BatchNormalization()(x)
    x = Activation(midlel_function)(x)
    x = Dropout(0.6)(x)

    x = Dense(4*scale)(normalized_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.6)(x)
  
    x = Dense(64)(normalized_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    output_layer = Dense(8)(x)
    output_layer = Dense(1, activation='sigmoid')(output_layer)

    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model with Huber loss function and AdamW optimizer
    # optimizer = AdamW(weight_decay=1e-4)
    # model.compile(loss=Huber(), optimizer=optimizer)    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])


    #model.compile(loss='mse', optimizer='adam')


    return model

In [8]:
def create_multipath_model(input_dim, output_dim, paths,layer1_size=2000):
    # Clear the session
    K.clear_session()

    # Create the model
    # Input layer
    input_layer = Input(shape=(input_dim,))

    # Batch normalization for input
    normalized_input = input_layer

    # Common path
    x = Dense(layer1_size)(normalized_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Define paths
    path_outputs = []
    for path in paths:
        activation_function = path[0]
        dropout_rate = path[1]
        layer_sizes = path[2:]
        path_output = x
        for size in layer_sizes:
            path_output = Dense(size)(path_output)
            path_output = BatchNormalization()(path_output)
            path_output = Activation(activation_function)(path_output)
            path_output = Dropout(dropout_rate)(path_output)
        path_outputs.append(path_output)

    # Concatenate paths
    concat = concatenate(path_outputs)

    # Output layer
    output_layer = Dense(256)(concat)
    output_layer = BatchNormalization()(output_layer)
    output_layer = Dense(128)(output_layer)
    output_layer = BatchNormalization()(output_layer)
    output_layer = Dense(64)(output_layer)
    output_layer = Dense(1, activation='sigmoid')(output_layer)

    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model with Huber loss function and AdamW optimizer
    optimizer = AdamW(weight_decay=1e-4)
    #model.compile(loss=Huber(), optimizer=optimizer)    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])


    #model.compile(loss='mse', optimizer='adam')


    return model

In [9]:
def create_minidirect_multipath_model(input_dim, output_dim, paths):
    # Clear the session
    K.clear_session()

    # Create the model
    # Input layer
    input_layer = Input(shape=(input_dim,))

    # Batch normalization for input
    normalized_input = input_layer


    x = normalized_input

    # Define paths
    path_outputs = []
    for path in paths:
        activation_function = path[0]
        dropout_rate = path[1]
        layer_sizes = path[2:]
        path_output = x
        for size in layer_sizes:
            path_output = Dense(size)(path_output)
            path_output = BatchNormalization()(path_output)
            path_output = Activation(activation_function)(path_output)
            path_output = Dropout(dropout_rate)(path_output)
        path_outputs.append(path_output)

    # Concatenate paths
    concat = concatenate(path_outputs)

    output_layer = Dense(64)(concat)
    output_layer = BatchNormalization()(output_layer)
    output_layer = Dense(29)(output_layer)
    output_layer = Dense(1, activation='sigmoid')(output_layer)
    

    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model with Huber loss function and AdamW optimizer
    # optimizer = AdamW(weight_decay=1e-4)
    #model.compile(loss=Huber(), optimizer=optimizer)    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])


    return model

In [10]:
from tensorflow.keras.regularizers import l1_l2

def create_minidirect_multipath_l1l2reg_model(input_dim, output_dim, paths, normalizer):
    # Clear the session
    K.clear_session()

    # Create the model
    # Input layer
    input_layer = Input(shape=(input_dim,))

    # Batch normalization for input
    normalized_input = input_layer

    x = normalized_input

    # Define paths
    path_outputs = []
    for path in paths:
        activation_function = path[0]
        dropout_rate = path[1]
        layer_sizes = path[2:]
        path_output = x
        for size in layer_sizes:
            path_output = Dense(size, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(path_output)
            path_output = BatchNormalization()(path_output)
            path_output = Activation(activation_function)(path_output)
            path_output = Dropout(dropout_rate)(path_output)
        path_outputs.append(path_output)

    # Concatenate paths
    concat = concatenate(path_outputs)

    output_layer = Dense(64, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(concat)
    output_layer = BatchNormalization()(output_layer)
    output_layer = Dense(29, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(output_layer)
    output_layer = Dense(1, activation='linear', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(output_layer)

    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model with Huber loss function and AdamW optimizer
    # optimizer = AdamW(weight_decay=1e-4)
    #model.compile(loss=Huber(), optimizer=optimizer)    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])


    return model

In [11]:
def create_multipathencoder_cnn(input_dim, output_dim, size, layer1_size=2000):
    # Clear the session
    K.clear_session()

    # Create the model
    # Input layer
    input_layer = Input(shape=(input_dim,))

    # Batch normalization for input
    normalized_input = input_layer

    # Common path
    x = Dense(layer1_size)(normalized_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.4)(x)
    paths = size*[['relu', 0.4, size]]
    
    # Define paths
    path_outputs = []
    for path in paths:
        activation_function = path[0]
        dropout_rate = path[1]
        layer_sizes = path[2:]
        path_output = x
        for size in layer_sizes:
            path_output = Dense(size)(path_output)
            path_output = BatchNormalization()(path_output)
            path_output = Activation(activation_function)(path_output)
            path_output = Dropout(dropout_rate)(path_output)
        path_outputs.append(path_output)

    # Concatenate paths
    concat = concatenate(path_outputs)
    reshape = Reshape((size, size, 1))(concat)  # Reshape to (size, size, 1) to fit CNN input shape

    # Triple-layer CNN
    conv = Conv2D(64, 3, padding='same')(reshape)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(64, 3, padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(64, 3, padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    pool = GlobalMaxPooling2D()(conv)

    # Output layer
    output_layer = Dense(196)(pool)
    output_layer = BatchNormalization()(output_layer)
    output_layer = Dense(64)(output_layer)
    output_layer = Dense(1, activation='sigmoid')(output_layer)

    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model with Huber loss function and AdamW optimizer
    # optimizer = AdamW(weight_decay=1e-4)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])


    return model

In [43]:
# Initialize a Sequential model
from tensorflow.keras.metrics import AUC,Precision, Recall


from keras.layers import BatchNormalization

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='softplus'),
    BatchNormalization(),
    Dropout(0.6),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.7),
    Dense(8, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])


In [44]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 128)               35456     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dropout_18 (Dropout)        (None, 128)               0         
                                                                 
 dense_33 (Dense)            (None, 128)               16512     
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dropout_19 (Dropout)        (None, 128)              

In [45]:
#Neral Network rulu 128 - sigmoid 1 :  accuracy: 0.8977 - auc_pr: 0.9452 - precision: 0.8667 - recall: 0.9401 - val_loss: 1.4164 - val_accuracy: 0.7669 - val_auc_pr: 0.8374 - val_precision: 0.8515 - val_recall: 0.6464
# relu 128 64 8- sigmoid 1 :  loss: 0.2566 - accuracy: 0.8933 - auc_pr: 0.9432 - precision_1: 0.8582 - recall_1: 0.9424 - val_loss: 0.5065 - val_accuracy: 0.8888 - val_auc_pr: 0.8797 - val_precision_1: 0.8719 - val_recall_1: 0.9116
# relu 128 64 - sigmoid 1 : loss: 0.2392 - accuracy: 0.9009 - auc_pr: 0.9505 - precision: 0.8639 - recall: 0.9518 - val_loss: 0.9174 - val_accuracy: 0.8723 - val_auc_pr: 0.8569 - val_precision: 0.8740 - val_recall: 0.8700
# relu 3*256 - sigmoid 1 : Best Index: 33, Best Accuracy: 0.8907231688499451, Best AUC PR: 0.8968896269798279, Best Precision: 0.8906420469284058, Best Recall: 0.8908270001411438
# gelu 3*256 sigmoid 1: Best Index: 7, Best Accuracy: 0.8779525756835938, Best AUC PR: 0.9052767753601074, Best Precision: 0.8704900741577148, Best Recall: 0.8880236744880676
# relu softplus 128 sigmoid 1: Best Index: 20, Best Accuracy: 0.8921507596969604, Best AUC PR: 0.8492059111595154, Best Precision: 0.8752608299255371, Best Recall: 0.9146550297737122
print("bn")
history = model.fit(X_train_scaled, y_train, epochs=60, batch_size=128, verbose=1, validation_data=(X_valid_scaled, y_valid))

bn
Epoch 1/60
6471/6471 [==============================] - 144s 22ms/step - loss: 0.4093 - accuracy: 0.8191 - auc_pr: 0.8662 - precision_7: 0.7892 - recall_7: 0.8708 - val_loss: 0.3214 - val_accuracy: 0.8586 - val_auc_pr: 0.9446 - val_precision_7: 0.8267 - val_recall_7: 0.9075
Epoch 2/60
6471/6471 [==============================] - 135s 21ms/step - loss: 0.3435 - accuracy: 0.8543 - auc_pr: 0.9058 - precision_7: 0.8274 - recall_7: 0.8953 - val_loss: 0.3355 - val_accuracy: 0.8679 - val_auc_pr: 0.9284 - val_precision_7: 0.8567 - val_recall_7: 0.8836
Epoch 3/60
6471/6471 [==============================] - 133s 20ms/step - loss: 0.3213 - accuracy: 0.8647 - auc_pr: 0.9175 - precision_7: 0.8405 - recall_7: 0.9001 - val_loss: 0.3156 - val_accuracy: 0.8813 - val_auc_pr: 0.9386 - val_precision_7: 0.8698 - val_recall_7: 0.8968
Epoch 4/60
6471/6471 [==============================] - 132s 20ms/step - loss: 0.3079 - accuracy: 0.8707 - auc_pr: 0.9241 - precision_7: 0.8469 - recall_7: 0.9049 - val_los

In [42]:
import numpy as np
best_index = np.argmax(history.history['val_accuracy'])
best_accuracy = history.history['val_accuracy'][best_index]
best_auc_pr = history.history['val_auc_pr'][best_index]
best_precision = history.history['val_precision_6'][best_index]
best_recall = history.history['val_recall_6'][best_index]
print(f"Best Index: {best_index}, Best Accuracy: {best_accuracy}, Best AUC PR: {best_auc_pr}, Best Precision: {best_precision}, Best Recall: {best_recall}")


Best Index: 20, Best Accuracy: 0.8921507596969604, Best AUC PR: 0.8492059111595154, Best Precision: 0.8752608299255371, Best Recall: 0.9146550297737122


In [30]:
best_index

33

In [26]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Define the function
def create_minimum_ffn(input_dim, output_dim, scale=1, middle_function='sigmoid'):
    # Clear the session
    K.clear_session()
    layer1_size=16
    # Create the model
    # Input layer
    input_layer = Input(shape=(input_dim,))

    # Batch normalization for input
    normalized_input = input_layer

    x = Dense(layer1_size*scale)(normalized_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.7)(x)

    x = Dense(8*scale)(normalized_input)
    x = BatchNormalization()(x)
    x = Activation(middle_function)(x)
    x = Dropout(0.6)(x)

    x = Dense(4*scale)(normalized_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.6)(x)
  
    x = Dense(64)(normalized_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    output_layer = Dense(8)(x)
    output_layer = Dense(1, activation='sigmoid')(output_layer)

    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])


    return model

# Create the model
#def create_minimum_ffn(input_dim, output_dim, scale=1, middle_function='sigmoid'):

model = create_minimum_ffn(X_train.shape[1], 1)

# Fit the model
model.fit(X_train_scaled, y_train, epochs=20, batch_size=64, verbose=1, validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
12942/12942 [==============================] - 180s 14ms/step - loss: 0.4002 - accuracy: 0.8249 - auc_pr: 0.8770 - precision: 0.7905 - recall: 0.8842 - val_loss: 0.3608 - val_accuracy: 0.8402 - val_auc_pr: 0.9075 - val_precision: 0.8010 - val_recall: 0.9053
Epoch 2/20
12942/12942 [==============================] - 180s 14ms/step - loss: 0.3627 - accuracy: 0.8449 - auc_pr: 0.9003 - precision: 0.8099 - recall: 0.9013 - val_loss: 0.4032 - val_accuracy: 0.8003 - val_auc_pr: 0.8978 - val_precision: 0.8011 - val_recall: 0.7990
Epoch 3/20
12942/12942 [==============================] - 180s 14ms/step - loss: 0.3491 - accuracy: 0.8517 - auc_pr: 0.9074 - precision: 0.8164 - recall: 0.9074 - val_loss: 0.4449 - val_accuracy: 0.7987 - val_auc_pr: 0.8975 - val_precision: 0.8132 - val_recall: 0.7755
Epoch 4/20
12942/12942 [==============================] - 175s 14ms/step - loss: 0.3422 - accuracy: 0.8543 - auc_pr: 0.9104 - precision: 0.8187 - recall: 0.9102 - val_loss: 0.4237 - val_accurac

In [38]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Define the function
def create_minidirect_multipath_model(input_dim, output_dim, paths):
    # Clear the session
    K.clear_session()

    # Create the model
    # Input layer
    input_layer = Input(shape=(input_dim,))

    # Batch normalization for input
    normalized_input = input_layer


    x = normalized_input

    # Define paths
    path_outputs = []
    for path in paths:
        activation_function = path[0]
        dropout_rate = path[1]
        layer_sizes = path[2:]
        path_output = x
        for size in layer_sizes:
            path_output = Dense(size)(path_output)
            path_output = BatchNormalization()(path_output)
            path_output = Activation(activation_function)(path_output)
            path_output = Dropout(dropout_rate)(path_output)
        path_outputs.append(path_output)

    # Concatenate paths
    concat = concatenate(path_outputs)

    output_layer = Dense(64)(concat)
    output_layer = BatchNormalization()(output_layer)
    output_layer = Dropout(dropout_rate)(output_layer)
    output_layer = Dense(4)(output_layer)
    output_layer = Dense(1, activation='sigmoid')(output_layer)
    

    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model with Huber loss function and AdamW optimizer
    # optimizer = AdamW(weight_decay=1e-4)
    #model.compile(loss=Huber(), optimizer=optimizer)    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])


    return model


In [39]:

#def create_minidirect_multipath_model(input_dim, output_dim, paths):
paths=[['relu', 0.5, 32, 8]]*6#, ['relu', 0.5, 16, 8], ['relu', 0.2, 8, 4],['relu', 0.5,64,16, 8, 4],['relu', 0.5,64,16, 8, 4]]
model = create_minidirect_multipath_model(X_train.shape[1], 1,paths)

# Fit the model
model.fit(X_train_scaled, y_train, epochs=20, batch_size=128, verbose=1, validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
6471/6471 [==============================] - 252s 38ms/step - loss: 0.4125 - accuracy: 0.8185 - auc_pr: 0.8673 - precision: 0.7840 - recall: 0.8793 - val_loss: 0.3169 - val_accuracy: 0.8589 - val_auc_pr: 0.9510 - val_precision: 0.8121 - val_recall: 0.9340
Epoch 2/20
6471/6471 [==============================] - 242s 37ms/step - loss: 0.3556 - accuracy: 0.8487 - auc_pr: 0.9019 - precision: 0.8180 - recall: 0.8970 - val_loss: 0.3169 - val_accuracy: 0.8676 - val_auc_pr: 0.9376 - val_precision: 0.8291 - val_recall: 0.9260
Epoch 3/20
6471/6471 [==============================] - 243s 38ms/step - loss: 0.3337 - accuracy: 0.8591 - auc_pr: 0.9133 - precision: 0.8299 - recall: 0.9034 - val_loss: 0.3307 - val_accuracy: 0.8581 - val_auc_pr: 0.9150 - val_precision: 0.8315 - val_recall: 0.8981
Epoch 4/20
6471/6471 [==============================] - 242s 37ms/step - loss: 0.3197 - accuracy: 0.8650 - auc_pr: 0.9202 - precision: 0.8357 - recall: 0.9086 - val_loss: 0.3439 - val_accuracy: 0.864

In [36]:
paths=[['relu', 0.4, 32,16, 8]]*3#, ['relu', 0.5, 16, 8], ['relu', 0.2, 8, 4],['relu', 0.5,64,16, 8, 4],['relu', 0.5,64,16, 8, 4]]
model = create_minidirect_multipath_model(X_train.shape[1], 1,paths)

# Fit the model
model.fit(X_train_scaled, y_train, epochs=20, batch_size=128, verbose=1, validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
6471/6471 [==============================] - 219s 33ms/step - loss: 0.4130 - accuracy: 0.8193 - auc_pr: 0.8673 - precision: 0.7877 - recall: 0.8744 - val_loss: 0.3565 - val_accuracy: 0.8494 - val_auc_pr: 0.9210 - val_precision: 0.8162 - val_recall: 0.9019
Epoch 2/20
6471/6471 [==============================] - 210s 32ms/step - loss: 0.3567 - accuracy: 0.8490 - auc_pr: 0.9010 - precision: 0.8224 - recall: 0.8904 - val_loss: 0.3681 - val_accuracy: 0.8549 - val_auc_pr: 0.8917 - val_precision: 0.8321 - val_recall: 0.8892
Epoch 3/20
6471/6471 [==============================] - 209s 32ms/step - loss: 0.3372 - accuracy: 0.8578 - auc_pr: 0.9117 - precision: 0.8318 - recall: 0.8971 - val_loss: 0.3471 - val_accuracy: 0.8683 - val_auc_pr: 0.9111 - val_precision: 0.8378 - val_recall: 0.9135
Epoch 4/20
6471/6471 [==============================] - 212s 33ms/step - loss: 0.3263 - accuracy: 0.8631 - auc_pr: 0.9163 - precision: 0.8368 - recall: 0.9020 - val_loss: 0.3750 - val_accuracy: 0.867

In [37]:
paths=[['relu', 0.5, 32,16, 8]]*5#, ['relu', 0.5, 16, 8], ['relu', 0.2, 8, 4],['relu', 0.5,64,16, 8, 4],['relu', 0.5,64,16, 8, 4]]
model = create_minidirect_multipath_model(X_train.shape[1], 1,paths)

# Fit the model
model.fit(X_train_scaled, y_train, epochs=20, batch_size=128, verbose=1, validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
6471/6471 [==============================] - 293s 44ms/step - loss: 0.4248 - accuracy: 0.8126 - auc_pr: 0.8571 - precision: 0.7758 - recall: 0.8792 - val_loss: 0.3468 - val_accuracy: 0.8432 - val_auc_pr: 0.9386 - val_precision: 0.8012 - val_recall: 0.9129
Epoch 2/20
6471/6471 [==============================] - 279s 43ms/step - loss: 0.3684 - accuracy: 0.8434 - auc_pr: 0.8935 - precision: 0.8123 - recall: 0.8932 - val_loss: 0.3413 - val_accuracy: 0.8515 - val_auc_pr: 0.9486 - val_precision: 0.8115 - val_recall: 0.9156
Epoch 3/20
6471/6471 [==============================] - 280s 43ms/step - loss: 0.3471 - accuracy: 0.8535 - auc_pr: 0.9055 - precision: 0.8238 - recall: 0.8993 - val_loss: 0.3375 - val_accuracy: 0.8570 - val_auc_pr: 0.9384 - val_precision: 0.8281 - val_recall: 0.9010
Epoch 4/20
6471/6471 [==============================] - 278s 43ms/step - loss: 0.3335 - accuracy: 0.8592 - auc_pr: 0.9127 - precision: 0.8301 - recall: 0.9034 - val_loss: 0.3629 - val_accuracy: 0.857

In [23]:
batch_sizes = [1024, 512, 256, 128, 64, 32]
results_df2 = pd.DataFrame(columns=["Batch Size", "Accuracy", "Loss", "Val_Accuracy", "AUC_PR", "Precision", "Recall"])

for batch_size in batch_sizes:
    history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=batch_size, verbose=1, validation_data=(X_valid_scaled, y_valid))
    accuracy = history.history['accuracy'][-1]
    loss = history.history['loss'][-1]
    val_accuracy = history.history['val_accuracy'][-1]
    auc_pr = history.history['val_auc_pr'][-1]
    precision = history.history['val_precision'][-1]
    recall = history.history['val_recall'][-1]
    new_row = pd.DataFrame({
        "Batch Size": [batch_size],
        "Accuracy": [accuracy],
        "Loss": [loss],
        "Val_Accuracy": [val_accuracy],
        "AUC_PR": [auc_pr],
        "Precision": [precision],
        "Recall": [recall]
    })
    results_df2 = pd.concat([results_df2, new_row], ignore_index=True)

Epoch 1/20
809/809 [==============================] - 4s 5ms/step - loss: 0.2328 - accuracy: 0.9045 - auc_pr: 0.9522 - precision: 0.8677 - recall: 0.9546 - val_loss: 0.7155 - val_accuracy: 0.8725 - val_auc_pr: 0.8344 - val_precision: 0.8783 - val_recall: 0.8648
Epoch 2/20
809/809 [==============================] - 4s 5ms/step - loss: 0.2316 - accuracy: 0.9054 - auc_pr: 0.9528 - precision: 0.8688 - recall: 0.9551 - val_loss: 0.7230 - val_accuracy: 0.8703 - val_auc_pr: 0.8568 - val_precision: 0.8724 - val_recall: 0.8676
Epoch 3/20
809/809 [==============================] - 4s 5ms/step - loss: 0.2300 - accuracy: 0.9062 - auc_pr: 0.9533 - precision: 0.8694 - recall: 0.9560 - val_loss: 0.7847 - val_accuracy: 0.8459 - val_auc_pr: 0.8638 - val_precision: 0.8787 - val_recall: 0.8026
Epoch 4/20
809/809 [==============================] - 4s 5ms/step - loss: 0.2282 - accuracy: 0.9070 - auc_pr: 0.9533 - precision: 0.8706 - recall: 0.9562 - val_loss: 0.7663 - val_accuracy: 0.8741 - val_auc_pr: 0.84

C:\Users\ayoub\AppData\Local\Temp\ipykernel_8904\1314869960.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df2 = pd.concat([results_df2, new_row], ignore_index=True)


Epoch 1/20
1618/1618 [==============================] - 7s 5ms/step - loss: 0.2256 - accuracy: 0.9092 - auc_pr: 0.9539 - precision: 0.8734 - recall: 0.9572 - val_loss: 1.0389 - val_accuracy: 0.8192 - val_auc_pr: 0.8387 - val_precision: 0.8745 - val_recall: 0.7454
Epoch 2/20
1618/1618 [==============================] - 8s 5ms/step - loss: 0.2213 - accuracy: 0.9112 - auc_pr: 0.9553 - precision: 0.8758 - recall: 0.9584 - val_loss: 1.1448 - val_accuracy: 0.8016 - val_auc_pr: 0.8180 - val_precision: 0.8768 - val_recall: 0.7017
Epoch 3/20
1618/1618 [==============================] - 8s 5ms/step - loss: 0.2200 - accuracy: 0.9119 - auc_pr: 0.9554 - precision: 0.8767 - recall: 0.9586 - val_loss: 1.1198 - val_accuracy: 0.8260 - val_auc_pr: 0.8185 - val_precision: 0.8813 - val_recall: 0.7536
Epoch 4/20
1618/1618 [==============================] - 8s 5ms/step - loss: 0.2191 - accuracy: 0.9120 - auc_pr: 0.9560 - precision: 0.8770 - recall: 0.9585 - val_loss: 1.0403 - val_accuracy: 0.8228 - val_auc_

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [25]:
results_df2.sort_values(by='AUC_PR', ascending=False)

,Batch Size,Accuracy,Loss,Val_Accuracy,AUC_PR,Precision,Recall
3,128,0.923915,0.198803,0.813243,0.846187,0.901731,0.703110
0,1024,0.915880,0.212051,0.817292,0.824375,0.881952,0.732648
1,512,0.919186,0.205489,0.790869,0.818765,0.886254,0.667393
2,256,0.922450,0.200001,0.788870,0.814965,0.889807,0.659399
4,64,0.924561,0.198726,0.775554,0.810235,0.895529,0.623890


In [42]:
history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=128, verbose=1, validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
6471/6471 [==============================] - 34s 5ms/step - loss: 0.3954 - accuracy: 0.8241 - auc_pr: 0.8788 - precision_1: 0.7825 - recall_1: 0.8976 - val_loss: 0.3372 - val_accuracy: 0.8581 - val_auc_pr: 0.9348 - val_precision_1: 0.8049 - val_recall_1: 0.9453
Epoch 2/20
6471/6471 [==============================] - 32s 5ms/step - loss: 0.3351 - accuracy: 0.8534 - auc_pr: 0.9130 - precision_1: 0.8153 - recall_1: 0.9138 - val_loss: 0.3871 - val_accuracy: 0.8687 - val_auc_pr: 0.8882 - val_precision_1: 0.8187 - val_recall_1: 0.9470
Epoch 3/20
6471/6471 [==============================] - 31s 5ms/step - loss: 0.3124 - accuracy: 0.8641 - auc_pr: 0.9239 - precision_1: 0.8256 - recall_1: 0.9233 - val_loss: 0.3976 - val_accuracy: 0.8739 - val_auc_pr: 0.8904 - val_precision_1: 0.8323 - val_recall_1: 0.9365
Epoch 4/20
6471/6471 [==============================] - 32s 5ms/step - loss: 0.2996 - accuracy: 0.8702 - auc_pr: 0.9294 - precision_1: 0.8319 - recall_1: 0.9278 - val_loss: 0.4024 -

In [6]:
# Initialize a Sequential model
from tensorflow.keras.metrics import AUC,Precision, Recall


model2 = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])

history = model2.fit(X_train_scaled, y_train, epochs=20, batch_size=128, verbose=1, validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
6471/6471 [==============================] - 36s 5ms/step - loss: 0.4181 - accuracy: 0.8165 - auc_pr: 0.8613 - precision: 0.7774 - recall: 0.8869 - val_loss: 0.3560 - val_accuracy: 0.8535 - val_auc_pr: 0.9404 - val_precision: 0.8023 - val_recall: 0.9383
Epoch 2/20
6471/6471 [==============================] - 34s 5ms/step - loss: 0.3559 - accuracy: 0.8451 - auc_pr: 0.8981 - precision: 0.8109 - recall: 0.9001 - val_loss: 0.3605 - val_accuracy: 0.8674 - val_auc_pr: 0.9252 - val_precision: 0.8182 - val_recall: 0.9447
Epoch 3/20
6471/6471 [==============================] - 34s 5ms/step - loss: 0.3328 - accuracy: 0.8570 - auc_pr: 0.9106 - precision: 0.8202 - recall: 0.9144 - val_loss: 0.3758 - val_accuracy: 0.8726 - val_auc_pr: 0.9287 - val_precision: 0.8276 - val_recall: 0.9412
Epoch 4/20
6471/6471 [==============================] - 35s 5ms/step - loss: 0.3192 - accuracy: 0.8634 - auc_pr: 0.9170 - precision: 0.8269 - recall: 0.9193 - val_loss: 0.3778 - val_accuracy: 0.8723 - val_

In [7]:
# Initialize a Sequential model
from tensorflow.keras.metrics import AUC,Precision, Recall


model2 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(96, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])

history = model2.fit(X_train_scaled, y_train, epochs=20, batch_size=128, verbose=1, validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
6471/6471 [==============================] - 37s 6ms/step - loss: 0.4008 - accuracy: 0.8250 - auc_pr: 0.8733 - precision_1: 0.7871 - recall_1: 0.8909 - val_loss: 0.3429 - val_accuracy: 0.8687 - val_auc_pr: 0.9478 - val_precision_1: 0.8229 - val_recall_1: 0.9397
Epoch 2/20
6471/6471 [==============================] - 35s 5ms/step - loss: 0.3338 - accuracy: 0.8567 - auc_pr: 0.9107 - precision_1: 0.8227 - recall_1: 0.9093 - val_loss: 0.3829 - val_accuracy: 0.8796 - val_auc_pr: 0.9287 - val_precision_1: 0.8418 - val_recall_1: 0.9350
Epoch 3/20
6471/6471 [==============================] - 34s 5ms/step - loss: 0.3084 - accuracy: 0.8685 - auc_pr: 0.9229 - precision_1: 0.8337 - recall_1: 0.9206 - val_loss: 0.4793 - val_accuracy: 0.8809 - val_auc_pr: 0.9041 - val_precision_1: 0.8522 - val_recall_1: 0.9216
Epoch 4/20
6471/6471 [==============================] - 35s 5ms/step - loss: 0.2921 - accuracy: 0.8762 - auc_pr: 0.9298 - precision_1: 0.8402 - recall_1: 0.9291 - val_loss: 0.5086 -

In [9]:
from tensorflow.keras.layers import BatchNormalization,Activation

model3 = Sequential([
    Dense(256, input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    
    Dense(128),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    
    Dense(96),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    
    Dense(64),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
])
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',AUC(curve='PR', name='auc_pr'),Precision(), Recall()])

history = model3.fit(X_train_scaled, y_train, epochs=20, batch_size=128, verbose=1, validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
6471/6471 [==============================] - 57s 8ms/step - loss: 0.3964 - accuracy: 0.8280 - auc_pr: 0.8746 - precision_2: 0.7954 - recall_2: 0.8832 - val_loss: 0.3246 - val_accuracy: 0.8663 - val_auc_pr: 0.9400 - val_precision_2: 0.8242 - val_recall_2: 0.9312
Epoch 2/20
6471/6471 [==============================] - 55s 8ms/step - loss: 0.3325 - accuracy: 0.8608 - auc_pr: 0.9109 - precision_2: 0.8306 - recall_2: 0.9064 - val_loss: 0.3517 - val_accuracy: 0.8716 - val_auc_pr: 0.9196 - val_precision_2: 0.8379 - val_recall_2: 0.9216
Epoch 3/20
6471/6471 [==============================] - 55s 9ms/step - loss: 0.3080 - accuracy: 0.8716 - auc_pr: 0.9221 - precision_2: 0.8403 - recall_2: 0.9178 - val_loss: 0.4065 - val_accuracy: 0.8492 - val_auc_pr: 0.9011 - val_precision_2: 0.8461 - val_recall_2: 0.8537
Epoch 4/20
6471/6471 [==============================] - 55s 8ms/step - loss: 0.2918 - accuracy: 0.8786 - auc_pr: 0.9293 - precision_2: 0.8470 - recall_2: 0.9241 - val_loss: 0.4219 -

In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f'Test accuracy: {accuracy:.3f}')


In [ ]:

# Make predictions
y_pred = model.predict_classes(X_test_scaled)

# Generate a classification report
report = classification_report(y_test, y_pred)
print(report)





## Other machine learning methodes

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, auc, precision_recall_curve
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np

# Define the models to benchmark
models = [
    ("Logistic Regression", LogisticRegression(max_iter=1000)),
    ("SVM", SVC(probability=True)),
    ("K-Nearest Neighbors", KNeighborsClassifier()),
    ("Decision Tree", DecisionTreeClassifier()),
    ("Random Forest", RandomForestClassifier()),
    ("Gradient Boosting", GradientBoostingClassifier()),
    ("MLP Classifier", MLPClassifier(max_iter=1000))
]

# Function to calculate AUC-PR
def calculate_auc_pr(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)

# Evaluate each model
results = []
for name, model in models:
    # Fit the model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_valid_scaled)
    y_pred_proba = model.predict_proba(X_valid_scaled)[:,1] if hasattr(model, "predict_proba") else model.decision_function(X_valid_scaled)
    
    # Calculate metrics
    accuracy = accuracy_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred)
    recall = recall_score(y_valid, y_pred)
    auc_pr = calculate_auc_pr(y_valid, y_pred_proba)
    
    # Store the results
    results.append((name, accuracy, precision, recall, auc_pr))

# Display the results
for name, accuracy, precision, recall, auc_pr in results:
    print(f"{name}: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, AUC-PR={auc_pr:.4f}")
